In [3]:
#precisa instalar alguns pacotes.

#NO MAC
'''
brew install tcl-tk
pip install lxml
brew install protobuf
brew install pil




'''
'''
# For CPU
!pip install tensorflow
# For GPU
!pip install tensorflow-gpu
!pip install --user Cython
!pip install --user contextlib2
!pip install lxml
!pip install PIL

!pip install --user Cython
!pip install --user contextlib2
!pip install --user pillow
!pip install --user lxml



#Precisa configurar o COCOAPI
!git clone https://github.com/cocodataset/cocoapi.git


#parece que ele precisa links direto para o folder onde está as funcoes da COCOAPI
!cp object_detection_tutorial.ipynb  /Users/arnaldoljr/.virtualenvs/cv/lib/python3.7/site-packages/tensorflow/models/research/object_detection/


OBSERVACAO

Any model exported using the export_inference_graph.py tool can be loaded here simply 
by changing PATH_TO_FROZEN_GRAPH to point to a new .pb file.
By default we use an "SSD with Mobilenet" model here. See the detection model zoo
for a list of other models that can be run out-of-the-box with varying speeds and accuracies.



'''
# This is needed to display the images.
%matplotlib inline
%pylab inline 


import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
from IPython.display import clear_output

import matplotlib.pyplot as plt

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#tem que carregar da pasta correta...

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

from utils import label_map_util

from utils import visualization_utils as vis_util




    # What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')


opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

def get_current_position(c):
    return int(c.get(0))
    #return int(c.get(CAP_PROP_POS_MSEC)/1000)
        
        
        
#MÉTODO otimizado de carregar o tensorflow sem muito custo...
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
#fazer o mapemento do label que ele dedecta para o que realmente é....tipo...label 5 quer dizer um 'carro' por exemplo

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


#funcao para otimizar o a entrada..
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

#funcao para puxar a webcam

# Using OpenCV to initialize the webcam
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('dashcam2.mp4')

with detection_graph.as_default():
    config = tf.ConfigProto(log_device_placement=False)
    config.gpu_options.allow_growth = False
    with tf.Session(graph=detection_graph, config=config) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
        while cap.isOpened():
            ret, image_np = cap.read()
            image_np_expanded = np.expand_dims(image_np, axis=0)

            (boxes, scores, classes, num) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections],feed_dict={image_tensor: image_np_expanded})

            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=10)
                #orig
                #cv2.imshow('MobileNet SSD - Object Detection', image_np)
                #teste1
                #plt.imshow(image_np)
                #plt.show()
                #teste2
                # Convert the image from OpenCV BGR format to matplotlib RGB format
                # to display the image
                
            #extraindo informações para o database
            frame = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
            millis = get_current_position(cap)
            #print(millis)
            #print("\n..............\n")
            '''
            data = {}
            i=0
            while ((output_dict['detection_scores'][i]) > 0.49999):
                chave = output_dict['detection_classes'][i]
                #print((output_dict['detection_scores'][i]))
                data.setdefault(chave,[]).append(output_dict['detection_scores'][i])
                i+=1


            print(data)    

            '''
            Coordenadasboxes= np.squeeze(boxes) 
            ClassesCoordenadas = np.squeeze(classes).astype(np.int32)
            Scoresdetectados = np.squeeze(scores)
            Indcateg = category_index
            with open('arqVideo.txt','a') as arq:
                arq.write("Tempo em milissegundos {} \n".format(millis)+"\n")
                arq.write("Coordenadas dections boxes: {} \n".format(Coordenadasboxes)+"\n")

                arq.write("Classes Coordenadas: {} \n".format(ClassesCoordenadas) +"\n")
                arq.write("Scores detectados: {} \n".format(Scoresdetectados)+"\n")
                arq.write("Índices de Categorias: {} \n".format(Indcateg)+"\n")

            arq.close()
           


            #print(scores)
            # Turn off the axis
            axis('off')
            # Title of the window
            title("Input Stream")
            # Display the frame
            imshow(frame)
            show()
            # Display the frame until new frame is available
            clear_output(wait=True)
        
            #if cv2.waitKey(1) == 13: #13 is the Enter Key
            #break

# Release camera and close windows
#cap.release()
#cv2.destroyAllWindows()  


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [ ]:
%html
<!-- TODO -->
<iframe width="960" height="415" src="" frameborder="0" allowfullscreen></iframe>

In [2]:
!pip install streamlink

     |▉                               | 10kB 8.9MB/s eta 0:00:01  Downloading https://files.pythonhosted.org/packages/e7/cc/bf41a8642ee9664b2140d909847632af7f13f326ece9c2bb379f4c0620df/streamlink-1.3.0-py2.py3-none-any.whl (384kB)
     |████████████████████████████████| 389kB 532kB/s eta 0:00:01
     |████████████████████████████████| 389kB 532kB/s eta 0:00:01
     |█████████▉                      | 61kB 1.3MB/s eta 0:00:011  Downloading https://files.pythonhosted.org/packages/29/19/44753eab1fdb50770ac69605527e8859468f3c0fd7dc5a76dd9c4dbd7906/websocket_client-0.56.0-py2.py3-none-any.whl (200kB)
     |████████████████████████████████| 204kB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 204kB 1.7MB/s eta 0:00:01
     |██                              | 10kB 12.0MB/s eta 0:00:01  Downloading https://files.pythonhosted.org/packages/5a/8d/27969852f4e664525c3d070e44b2b719bc195f4d18c311c52e57bb93614e/iso-639-0.4.5.tar.gz (167kB)
     |████████████████████████████████| 174kB 1.6MB

In [5]:
#teste lendo direto da url
import streamlink
import cv2

url = 'https://www.youtube.com/watch?v=bZ6NL59FMoc'

streams = streamlink.streams(url)
print(streams)
cap = cv2.VideoCapture(streams["360p"].url)
#ficar só no 360p para ficar mais rápido 
#cap = cv2.VideoCapture(streams["best"].url)

while True:
    ret, frame = cap.read()
    print(frame)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

OrderedDict([('audio_mp4', <HTTPStream('https://r1---sn-uxaxh5g-4vgl.googlevideo.com/videoplayback?expire=1576721442&ei=wof6Xfm2JeSBobIP1YaUSA&ip=2804%3Ad4b%3A7acb%3Ac800%3A75ca%3Ae50c%3Ab7e3%3A89c6&id=o-AHY7EF4A06DbbZHPCAzjwTQmqRD7dOXnVNRUHK5DZcgz&itag=140&source=youtube&requiressl=yes&mm=31%2C29&mn=sn-uxaxh5g-4vgl%2Csn-gpv7dn7e&ms=au%2Crdu&mv=m&mvi=0&pl=40&initcwndbps=480000&mime=audio%2Fmp4&gir=yes&clen=569708165&dur=35202.146&lmt=1572711118250676&mt=1576699707&fvip=6&keepalive=yes&fexp=23842630&c=WEB&txp=1301222&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&sig=ALgxI2wwRAIgCaiqvCLByKv8_69Qn7ci_2h7aKGGuN8aOtxeZgUJ14ICIHTfkVzTGswt5JpN2wFfua0EqX3ZcZ6KqjU4-Bxy6IVq&lsparams=mm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AHylml4wRAIgFe3jka14OEZLaNgQTcIc2dk4VS0SwRMCxHfPGtbgZggCIE40CuaFw8CzCjt8Co1lYjBwu5mLfTy23W448VNims9i')>), ('audio_webm', <HTTPStream('https://r1---sn-uxaxh5g-4vgl.googlevideo.com/videoplayback?expire=1576721442&ei=wof6Xfm2JeS

[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]]
[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ..

[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]]
[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ..

[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]]
[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ..

[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]]
[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ..

...

[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]]
[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ..

[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]]
[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ..

[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]]
[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ..

[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]]
[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ..

[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]]
[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [253 253 253]
  [253 253 253]
  [253 253 253]]

 ...

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ..

In [9]:
  
#teste da aplicacao lendo direto do youtube

    
    
#precisa instalar alguns pacotes.

#NO MAC
'''
brew install tcl-tk
pip install lxml
brew install protobuf
brew install pil




'''
'''
# For CPU
!pip install tensorflow
# For GPU
!pip install tensorflow-gpu
!pip install --user Cython
!pip install --user contextlib2
!pip install lxml
!pip install PIL

!pip install --user Cython
!pip install --user contextlib2
!pip install --user pillow
!pip install --user lxml



#Precisa configurar o COCOAPI
!git clone https://github.com/cocodataset/cocoapi.git


#parece que ele precisa links direto para o folder onde está as funcoes da COCOAPI
!cp object_detection_tutorial.ipynb  /Users/arnaldoljr/.virtualenvs/cv/lib/python3.7/site-packages/tensorflow/models/research/object_detection/


OBSERVACAO

Any model exported using the export_inference_graph.py tool can be loaded here simply 
by changing PATH_TO_FROZEN_GRAPH to point to a new .pb file.
By default we use an "SSD with Mobilenet" model here. See the detection model zoo
for a list of other models that can be run out-of-the-box with varying speeds and accuracies.



'''
# This is needed to display the images.
%matplotlib inline
%pylab inline 


import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
from IPython.display import clear_output
import streamlink
import matplotlib.pyplot as plt

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#tem que carregar da pasta correta...

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

from utils import label_map_util

from utils import visualization_utils as vis_util




    # What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')


opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

def get_current_position(c):
    return int(c.get(0))
    #return int(c.get(CAP_PROP_POS_MSEC)/1000)
        
        
        
#MÉTODO otimizado de carregar o tensorflow sem muito custo...
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
#fazer o mapemento do label que ele dedecta para o que realmente é....tipo...label 5 quer dizer um 'carro' por exemplo

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


#funcao para otimizar o a entrada..
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)


#funcao para puxar a webcam

# Using OpenCV to initialize the webcam
#cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('dashcam2.mp4')

#url = 'https://www.youtube.com/watch?v=bZ6NL59FMoc'

#videos para teste
url = 'https://www.youtube.com/watch?v=3a4n1RYMDbI'



'''

url = https://www.youtube.com/watch?v=y9t9_COpX_Y 

url = https://www.youtube.com/watch?v=VRlOKk32UnI 


'''

streams = streamlink.streams(url)
#print(streams)
cap = cv2.VideoCapture(streams["360p"].url)
#ficar só no 360p para ficar mais rápido 
#cap = cv2.VideoCapture(streams["best"].url)




with detection_graph.as_default():
    config = tf.ConfigProto(log_device_placement=False)
    config.gpu_options.allow_growth = False
    with tf.Session(graph=detection_graph, config=config) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
        while cap.isOpened():
            ret, image_np = cap.read()
            image_np_expanded = np.expand_dims(image_np, axis=0)

            (boxes, scores, classes, num) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections],feed_dict={image_tensor: image_np_expanded})

            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=10)
                #orig
                #cv2.imshow('MobileNet SSD - Object Detection', image_np)
                #teste1
                #plt.imshow(image_np)
                #plt.show()
                #teste2
                # Convert the image from OpenCV BGR format to matplotlib RGB format
                # to display the image
                
            #extraindo informações para o database
            frame = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
            millis = get_current_position(cap)
            #print(millis)
            #print("\n..............\n")
            '''
            data = {}
            i=0
            while ((output_dict['detection_scores'][i]) > 0.49999):
                chave = output_dict['detection_classes'][i]
                #print((output_dict['detection_scores'][i]))
                data.setdefault(chave,[]).append(output_dict['detection_scores'][i])
                i+=1


            print(data)    

            '''
            Coordenadasboxes= np.squeeze(boxes) 
            ClassesCoordenadas = np.squeeze(classes).astype(np.int32)
            Scoresdetectados = np.squeeze(scores)
            Indcateg = category_index
            with open('arqVideo.txt','a') as arq:
                arq.write("Tempo em milissegundos {} \n".format(millis)+"\n")
                arq.write("Coordenadas dections boxes: {} \n".format(Coordenadasboxes)+"\n")

                arq.write("Classes Coordenadas: {} \n".format(ClassesCoordenadas) +"\n")
                arq.write("Scores detectados: {} \n".format(Scoresdetectados)+"\n")
                arq.write("Índices de Categorias: {} \n".format(Indcateg)+"\n")

            arq.close()
           


            #print(scores)
            # Turn off the axis
            axis('off')
            # Title of the window
            title("Input Stream")
            # Display the frame
            imshow(frame)
            show()
            # Display the frame until new frame is available
            clear_output(wait=True)
        
            #if cv2.waitKey(1) == 13: #13 is the Enter Key
            #break

# Release camera and close windows
#cap.release()
#cv2.destroyAllWindows()  


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'